### ベースラインモデル (CNN + GRU)

In [2]:
import torch
import torch.nn as nn

class CNN_GRU_Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(CNN_GRU_Model, self).__init__()
        
        # 1. Embedding層
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # 2. CNN層 (1次元CNN)
        self.cnn = nn.Conv1d(in_channels=embedding_dim, out_channels=hidden_dim, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        
        # 3. GRU層
        self.gru = nn.GRU(input_size=hidden_dim, hidden_size=hidden_dim, batch_first=True)
        
        # 4. 全結合層
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # x の形状: (batch_size, seq_len)
        
        x = self.embedding(x)  # -> (batch_size, seq_len, embedding_dim)
        
        # CNNは (batch, channels, seq_len) の入力を期待するので、次元を入れ替える
        x = x.permute(0, 2, 1) # -> (batch_size, embedding_dim, seq_len)
        x = self.cnn(x)        # -> (batch_size, hidden_dim, seq_len)
        x = self.relu(x)
        
        # GRUの入力に合わせて再度次元を入れ替える
        x = x.permute(0, 2, 1) # -> (batch_size, seq_len, hidden_dim)
        _, h_n = self.gru(x)   # h_n の形状: (num_layers, batch_size, hidden_dim)
        
        # 最後の隠れ状態を使って分類
        x = h_n.squeeze(0)     # -> (batch_size, hidden_dim)
        out = self.fc(x)       # -> (batch_size, num_classes)
        
        return out

### pklデータの読み込み

In [ ]:
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# A. pklファイルを読み込む
with open('processed_data.pkl', 'rb') as f:
    data = pickle.load(f)

padded_ids = data['padded_ids'] # これはPythonのリスト
labels = data['labels']         # これもPythonのリスト

# B. Pythonリストの状態でデータを分割
# まずは訓練＋検証用とテスト用に分ける
train_val_ids, test_ids, train_val_labels, test_labels = train_test_split(
    padded_ids, labels, test_size=0.1, random_state=42, stratify=labels
)
# 次に訓練用と検証用に分ける
train_ids, val_ids, train_labels, val_labels = train_test_split(
    train_val_ids, train_val_labels, test_size=(1/9), random_state=42, stratify=train_val_labels
)

# D. Datasetクラスを定義（__getitem__でTensorに変換）
class LivedoorTensorDataset(Dataset):
    def __init__(self, ids, labels):
        self.ids = ids
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        # ここでPythonの数値リストからPyTorchのTensorに変換する
        input_ids = torch.tensor(self.ids[index], dtype=torch.long)
        label_id = torch.tensor(self.labels[index], dtype=torch.long)
        return input_ids, label_id

# C. 各データセットのインスタンスを作成
train_dataset = LivedoorTensorDataset(train_ids, train_labels)
val_dataset = LivedoorTensorDataset(val_ids, val_labels)
test_dataset = LivedoorTensorDataset(test_ids, test_labels)

# E. DataLoaderを作成
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# これでtrain_loaderから取り出されるデータはTensorのバッチになっている
# for batch_ids, batch_labels in train_loader:
#     print(batch_ids.shape)
#     break

### 学習・検証

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# ----------------------------------------------------
# <<< 追加点 (GPUの活用)
# ----------------------------------------------------
# GPUが利用可能か確認し、deviceを定義
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# モデルを定義し、指定したdeviceに転送
# model = CNN_GRU_Model(vocab_size, embedding_dim, hidden_dim, num_classes) # 例
model.to(device)

# ----------------------------------------------------
# <<< 修正点 (Optimizerの対象を全パラメータに)
# ----------------------------------------------------
# model.parameters()を渡すことで、モデルの全ての層のパラメータが学習対象になる
optimizer = optim.Adam(model.parameters(), lr=1e-4)

criterion = nn.CrossEntropyLoss()

# これまでの最小の検証損失を保存する変数を初期化（無限大で設定）
best_valid_loss = np.inf
MODEL_SAVE_PATH = 'best_model.pth' # 保存先のファイルパス

n_epochs = 7
for epoch in range(n_epochs):
    # 各エポックの損失と正解数を初期化
    loss_train = 0
    loss_valid = 0
    accuracy_train = 0
    accuracy_valid = 0

    # --- 訓練モード ---
    model.train()
    for batch in train_loader:
        # <<< 修正点 (データをdeviceに転送)
        x = batch["input_ids"].to(device)
        t = batch["label"].to(device)
        
        # 勾配をリセット
        optimizer.zero_grad()

        # 順伝播
        output = model(x)
        
        # 損失計算と逆伝播
        loss = criterion(output, t)
        loss.backward()
        
        # パラメータ更新
        optimizer.step()
        
        # 予測ラベルを計算
        pred = output.argmax(dim=1)

        # 損失と正解数を加算
        loss_train += loss.item()
        accuracy_train += torch.sum(pred == t.data)

    # --- 検証モード ---
    model.eval()
    # ----------------------------------------------------
    # <<< 修正点 (勾配計算を無効化)
    # ----------------------------------------------------
    with torch.no_grad():
        for batch in val_loader:
            # <<< 修正点 (データをdeviceに転送)
            x = batch["input_ids"].to(device)
            t = batch["label"].to(device)
            
            # 順伝播
            output = model(x)
            
            # 損失計算
            loss = criterion(output, t)
            
            # 予測ラベルを計算
            pred = output.argmax(dim=1)

            # 損失と正解数を加算
            loss_valid += loss.item()
            accuracy_valid += torch.sum(pred == t.data)
    
    # --- エポックごとの結果を計算・表示 ---
    # 損失はバッチ数の合計、正解率はデータ数の合計で割る
    avg_loss_train = loss_train / len(train_loader)
    avg_loss_valid = loss_valid / len(val_loader)
    avg_acc_train = accuracy_train / len(train_dataset)
    avg_acc_valid = accuracy_valid / len(val_dataset)

    # f-stringを使うと、より直感的に記述できる
    print(
        f"| epoch {epoch+1:2d} | train loss {avg_loss_train:.4f}, acc {avg_acc_train:.4f} "
        f"| valid loss {avg_loss_valid:.4f}, acc {avg_acc_valid:.4f}"
    )

    # ----------------------------------------------------
    # ----------------------------------------------------
    # 検証データの損失がこれまでの最小値を更新したら、モデルを保存
    if avg_loss_valid < best_valid_loss:
        print(f"Validation loss improved ({best_valid_loss:.4f} --> {avg_loss_valid:.4f}). Saving model...")
        best_valid_loss = avg_loss_valid # 最小値を更新
        
        # モデルのパラメータ（重み）のみを保存するのが推奨される方法
        torch.save(model.state_dict(), MODEL_SAVE_PATH)

print(f"\nTraining finished. Best model saved to {MODEL_SAVE_PATH}")

### 評価

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# --- 準備 ---
# 学習済みモデルをロードし、deviceに転送
# model.load_state_dict(torch.load('best_model.pth')) # 例
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 予測結果と正解ラベルを格納するリスト
all_preds = []
all_labels = []

# --- 評価モード ---
model.eval()
with torch.no_grad():
    for batch in test_loader: # test_loaderを使用
        # データをdeviceに転送
        x = batch["input_ids"].to(device)
        t = batch["label"].to(device)
        
        # モデルの出力を取得
        output = model(x)
        
        # 予測ラベルを計算 (argmaxで最も確率の高いクラスを取得)
        pred = output.argmax(dim=1)
        
        # 予測結果と正解ラベルをリストに追加
        # scikit-learnで計算するために、GPU上のTensorからCPU上のnumpy配列に変換
        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(t.cpu().numpy())

# --- 評価指標の計算 (scikit-learnを使用) ---
# 正解率
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy:.4f}")

# カテゴリ名のリスト（id_to_label辞書から取得）
# category_names = list(id_to_label.values()) # 例

# 適合率、再現率、F1スコアを含む詳細レポート
print("\nClassification Report:")
report = classification_report(all_labels, all_preds, target_names=category_names)
print(report)

# -------------テキストファイルへの保存--------------------
with open("test_results.txt", "w") as f:
f.write(f"Accuracy: {accuracy:.4f}\n")
f.write("\nClassification Report:\n")
f.write(report)

print("\nTest results saved to test_results.txt")

# --- 混同行列の可視化 ---
print("\nConfusion Matrix:")
cm = confusion_matrix(all_labels, all_preds)

# DataFrameに変換すると、軸ラベルが見やすくなる
cm_df = pd.DataFrame(cm, index=category_names, columns=category_names)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

plt.tight_layout() # ラベルが切れないように調整

#-------------混同行列の画像ファイルへの保存------------

plt.savefig("confusion_matrix.png")
print("\nConfusion matrix saved to confusion_matrix.png")

plt.show()